<a href="https://colab.research.google.com/github/FelipeSotoG/Desafio-1-EDA/blob/main/WTA_Hill_climbing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WTA

Problema WTA resuelto con Hill Climbing Algorithm

Initial state es ingresado manualmente.

In [9]:
from numpy.matrixlib.defmatrix import matrix
import numpy as np
import itertools
import copy
#formato de los estados
class State:
  id_iter = itertools.count()
  def __init__(self, matrix, weapons, prob, value):
    self.id = next(State.id_iter)
    self.matrix = matrix
    self.weapons = weapons
    self.prob = prob
    self.value = value

  def __lt__(self, other):
    return self.id < other.id
#matriz "vacía"
initial_state = State([[1,1,3],\
                       [1,1,0],\
                       [0,0,1]],
                       [5,2,1],
                      [[0.3,0.2,0.5],\
                       [0.1,0.6,0.5],\
                       [0.4,0.5,0.4]],
                       [5,10,20] )

class Action:
  def __init__(self,i,j):
    self.i=i; self.j=j

N=3 #número de filas/columnas

El metodo de transision es solo mover un item de la matriz a otra posicion, que se traduce a poner una arma a otro objetivo.

In [10]:
def transition(state, action):
  state.matrix[action.i[0]][action.i[1]] -=1 
  state.matrix[action.j[0]][action.j[1]] +=1 

Para ver las acciones validad se tiene que revizar la matriz para encontrar un punto que no sea 0, lo que indica que hay un item que mover.

In [11]:
def get_valid_actions(state):
  valid_actions = []
  for i in range(N):
    for j in range(N):
      if  state.matrix[i][j] != 0:
        for it in range(N):
          if it != j:
            valid_actions.append(Action([i,j],[i,it]))
  return valid_actions

Test de get_valid_actions()

In [12]:
for x in get_valid_actions(initial_state):
  print(x.i,x.j)
  new_state = copy.deepcopy(initial_state)
  transition(new_state,x)
  print(new_state.matrix)

[0, 0] [0, 1]
[[0, 2, 3], [1, 1, 0], [0, 0, 1]]
[0, 0] [0, 2]
[[0, 1, 4], [1, 1, 0], [0, 0, 1]]
[0, 1] [0, 0]
[[2, 0, 3], [1, 1, 0], [0, 0, 1]]
[0, 1] [0, 2]
[[1, 0, 4], [1, 1, 0], [0, 0, 1]]
[0, 2] [0, 0]
[[2, 1, 2], [1, 1, 0], [0, 0, 1]]
[0, 2] [0, 1]
[[1, 2, 2], [1, 1, 0], [0, 0, 1]]
[1, 0] [1, 1]
[[1, 1, 3], [0, 2, 0], [0, 0, 1]]
[1, 0] [1, 2]
[[1, 1, 3], [0, 1, 1], [0, 0, 1]]
[1, 1] [1, 0]
[[1, 1, 3], [2, 0, 0], [0, 0, 1]]
[1, 1] [1, 2]
[[1, 1, 3], [1, 0, 1], [0, 0, 1]]
[2, 2] [2, 0]
[[1, 1, 3], [1, 1, 0], [1, 0, 0]]
[2, 2] [2, 1]
[[1, 1, 3], [1, 1, 0], [0, 1, 0]]


## Resolución del problema

Ya modelado pordemos hacer la huristica que ocuparemos, la cual es la funcion objetivo.

Luego podemos implementar el algoritmo de hill climbing

In [13]:
import numpy as np

def heuristic_eval(state):
    minsum=0
    for j in range(N):
      arr=[]
      for i in range(N):
        if state.matrix[i][j] != 0: arr.append(pow(1-state.prob[i][j],state.matrix[i][j]))
      minsum+= state.value[j]*np.prod(arr) 
    return minsum

V1 tomando la primera mejora en la heuristica

In [14]:
def hill_climbing(initial_state):
  stack = [copy.deepcopy(initial_state)]
  iters = 0
  while len(stack)>0:
    iters += 1
    state = stack.pop()
    actions = get_valid_actions(state)
    minv= heuristic_eval(state)
    for action in actions:
      new_state = copy.deepcopy(state)
      transition(new_state,action)
      if heuristic_eval(new_state) < minv:
        stack.append(new_state)
        break
  return state,iters

In [15]:
state,iters=hill_climbing(initial_state)
print("==============================================")
print(state.matrix)
print(heuristic_eval(state))
print("iteraciones:", iters)

[[2, 0, 3], [0, 2, 0], [0, 0, 1]]
5.55
iteraciones: 3


V2 tomando la mejor de todas las acciones en la heutistica

In [16]:
def hill_climbing(initial_state):
  stack = [copy.deepcopy(initial_state)]
  iters = 0
  while len(stack)>0:
    iters += 1
    state = stack.pop()
    actions = get_valid_actions(state)
    minv= heuristic_eval(state)
    for action in actions:
      arr=[]
      new_state = copy.deepcopy(state)
      transition(new_state,action)
      if heuristic_eval(new_state) < minv:
        arr.append([heuristic_eval(new_state),new_state])
    if len(arr)!=0:
      arr.sort(reverse = True)
      stack.append(arr.pop()[1])
  return state,iters

In [17]:
state,iters=hill_climbing(initial_state)
print("==============================================")
print(state.matrix)
print(heuristic_eval(state))
print("iteraciones:", iters)

[[1, 1, 3], [1, 1, 0], [0, 1, 0]]
7.25
iteraciones: 2


Importante destacar el V2 funciona peor porque en este estado inicial, entra en un minimo local si es que se toman todas las acciones posibles en consideración.